In [1]:
import numpy as np
import os
import pandas as pd 

In [2]:
red_wine_file_path=r'T:\TOSHITH\PROGRAMMING\Wine-Quality-Classification-using-ML\winequality-red.csv'
white_wine_file_path=r'T:\TOSHITH\PROGRAMMING\Wine-Quality-Classification-using-ML\winequality-white.csv'

### step 1 combine data and preprocess it for red and white wine seperately 

In [3]:
red_wine_df=pd.read_csv(red_wine_file_path,sep=';')
white_wine_df=pd.read_csv(white_wine_file_path,sep=';')
'''
#unwanted_qualities = [ 3, 9,8,4]

# Filter out those rows from red and white wine data
red_wine_df = red_wine_df[~red_wine_df['quality'].isin(unwanted_qualities)]
white_wine_df = white_wine_df[~white_wine_df['quality'].isin(unwanted_qualities)]'''

print(red_wine_df.shape,white_wine_df.shape)
#red_wine_df.head()
print(red_wine_df['quality'].value_counts())
print(white_wine_df['quality'].value_counts())

(1599, 12) (4898, 12)
quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64
quality
6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: count, dtype: int64


So we see that the data is unbalanced 

In [4]:
from sklearn.preprocessing import StandardScaler
#function to scale our data to make it easier for our model to learn 

def scaler(wine_df):
    columns=list(wine_df.columns)

    wine_df=StandardScaler().fit_transform(wine_df)
    wine_df=pd.DataFrame(wine_df,columns=columns)
    return wine_df

'''is this really necessairy as some row values have become negative'''

'is this really necessairy as some row values have become negative'

### 1. Undersampling
* As white is more we will reduce it's numbers to equal reds
* Also we will do a train test split before sampling as we dont want to introduce redundant samples in the training and testing data 

In [5]:
from sklearn.model_selection import train_test_split
#70:30 split 
split_ratio=0.3
#the data frame in train and test still contains the quality column so we have garbage values which we will not use

red_wine_train, red_wine_test, garbage1,garbage2 = train_test_split(red_wine_df,red_wine_df['quality'],test_size=split_ratio)


#we want same no of samples in white wine test as red wine 

white_wine_test=white_wine_df.sample(n=red_wine_test.shape[0])
white_wine_train=white_wine_df.drop(white_wine_test.index)

red_wine_train.shape,red_wine_test.shape,white_wine_train.shape, white_wine_test.shape

((1119, 12), (480, 12), (4418, 12), (480, 12))

In [6]:
# here i am trying to get the same record composition in white test as red test 
# Get red wine test set distribution
quality_counts = red_wine_test['quality'].value_counts()

# Sample white wine test set with the same quality distribution
white_wine_test_list = []

for quality, count in quality_counts.items():
    subset = white_wine_df[white_wine_df['quality'] == quality]
    
    # Check if white wine has enough samples for the current quality
    if len(subset) < count:
        raise ValueError(f"Not enough white wine samples for quality {quality}")
    
    sampled = subset.sample(n=count, random_state=42)
    white_wine_test_list.append(sampled)

# Combine sampled groups into final white_wine_test
white_wine_test = pd.concat(white_wine_test_list)

# Drop these from the white_wine_df to get white_wine_train
white_wine_train = white_wine_df.drop(white_wine_test.index)

# Optional: print distribution comparison
print("Red Wine Test Distribution:\n", red_wine_test['quality'].value_counts().sort_index())
print("\nWhite Wine Test Distribution:\n", white_wine_test['quality'].value_counts().sort_index())

Red Wine Test Distribution:
 quality
3      4
4     22
5    198
6    191
7     59
8      6
Name: count, dtype: int64

White Wine Test Distribution:
 quality
3      4
4     22
5    198
6    191
7     59
8      6
Name: count, dtype: int64


encoding red as 0 and white as 1 

In [7]:
def return_train_and_test(red_wine_tr,white_wine_tr,red_wine_test,white_wine_test):
    # Add 'colour' column
    red_wine_tr['colour'] = 0
    white_wine_tr['colour'] = 1
    red_wine_test['colour'] = 0
    white_wine_test['colour'] = 1

    # Drop 'quality' for X, but keep it for Y
    X_train = pd.concat([
        red_wine_tr.drop(columns=['quality']),
        white_wine_tr.drop(columns=['quality'])
    ], axis=0)

    Y_train = pd.concat([
        red_wine_tr['quality'],
        white_wine_tr['quality']
    ], axis=0)

    X_test = pd.concat([
        red_wine_test.drop(columns=['quality']),
        white_wine_test.drop(columns=['quality'])
    ], axis=0)

    Y_test = pd.concat([
        red_wine_test['quality'],
        white_wine_test['quality']
    ], axis=0)

    return X_train,X_test,Y_train,Y_test

US is for under sample , N is for normal with out sampling 

In [8]:
#need to undersample white 
no_of_red=red_wine_train.shape[0] #shape of train y

N_X_train,N_X_test,N_Y_train,N_Y_test=return_train_and_test(red_wine_train,white_wine_train,red_wine_test,white_wine_test)

sample_white= white_wine_train.sample(no_of_red) # we get equal no of columns 
US_X_train,US_X_test,US_Y_train,US_Y_test=return_train_and_test(red_wine_train,sample_white,red_wine_test,white_wine_test)

US_X_train.shape,US_X_test.shape,US_Y_train.shape,US_Y_test.shape,N_X_train.shape,

((2238, 12), (960, 12), (2238,), (960,), (5537, 12))

In [9]:
US_X_train=scaler(US_X_train)
US_X_test=scaler(US_X_test)
#as the colour column has changed because of the standard scaler 
US_X_train['colour'] = US_X_train['colour'].replace(-1, 0)
US_X_test['colour'] = US_X_test['colour'].replace(-1, 0)

US_X_train.head(1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colour
0,-0.383219,1.524135,-1.395194,-0.457723,0.593153,-0.605964,-1.193349,0.110145,0.7891,0.166899,0.70689,0.0


In [10]:
N_X_train = scaler(N_X_train)
N_X_test = scaler(N_X_test)
# as these 2 are unbalenced data ill change the colour column to 0 and 1 manually 
#can apply this to above as well 

unique_values = N_X_train['colour'].unique()
lower_value = min(unique_values)

N_X_train['colour'] = (N_X_train['colour'] != lower_value).astype(int)

unique_values = N_X_test['colour'].unique()
lower_value = min(unique_values)
N_X_test['colour'] = (N_X_test['colour'] != lower_value).astype(int)

N_X_train.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colour
5532,-0.763330,-0.759210,-0.220608,-0.832715,-0.457760,-0.419812,-0.500299,-1.131484,0.354186,-0.168153,0.571565,1
5533,-0.443482,-0.052687,0.275306,0.495264,-0.213124,1.445371,0.886705,0.112275,-0.401426,-0.452991,-0.756327,1
5534,-0.523444,-0.566522,-0.929057,-0.915713,-0.396601,-0.080688,-0.153548,-0.668382,-1.408907,-0.452991,-0.922313,1
5535,-1.323065,-0.245375,-0.149763,-0.936463,-0.977613,-0.645895,-0.171798,-1.941912,0.794959,-1.022667,1.899456,1
5536,-0.923255,-0.759210,0.416996,-0.998712,-1.038773,-0.532853,-0.390799,-1.703746,0.291218,-1.449924,1.069524,1


In [ ]:
#perform train test split before sampling and sampling should be done only on training data 

#also verify if the data column has the same no of 1 and 0 (y values) values 11:57 in the video 


2 upsampling 

In [11]:
print(N_X_train['colour'].value_counts()) # w

colour
1    4418
0    1119
Name: count, dtype: int64


In [12]:
# Reset index to avoid index misalignment
N_X_train = N_X_train.reset_index(drop=True)
N_Y_train = N_Y_train.reset_index(drop=True)

# Split by colour
red_mask = N_X_train['colour'] == 0
red_X = N_X_train[red_mask]
red_Y = N_Y_train[red_mask]

white_mask = N_X_train['colour'] == 1
white_X = N_X_train[white_mask]
white_Y = N_Y_train[white_mask]

# Now you can upsample
from sklearn.utils import resample
red_X_upsampled, red_Y_upsampled = resample(
    red_X, red_Y,
    replace=True,
    n_samples=len(white_X),
    random_state=42
)

UP_X_train = pd.concat([red_X_upsampled, white_X])
UP_Y_train = pd.concat([red_Y_upsampled, white_Y])
UP_X_test = US_X_test.copy()
UP_Y_test = US_Y_test.copy()



In [13]:
print(UP_X_train['colour'].value_counts()) # w

colour
0    4418
1    4418
Name: count, dtype: int64


### apply smote 

In [15]:
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd

In [16]:
print("Original class distribution:", Counter(N_Y_train))

Original class distribution: Counter({6: 2454, 5: 1742, 7: 961, 8: 181, 4: 172, 3: 22, 9: 5})


In [17]:
smote = SMOTE(random_state=42)
sampling_strategy = {4: 500, 8: 500}
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
SM_X_train, SM_Y_train = smote.fit_resample(N_X_train, N_Y_train)
print("Resampled class distribution:", Counter(SM_Y_train))

Resampled class distribution: Counter({6: 2454, 5: 1742, 7: 961, 4: 500, 8: 500, 3: 22, 9: 5})


In [18]:
print(SM_X_train['colour'].value_counts()) #

colour
1    4986
0    1198
Name: count, dtype: int64


### use different ML model for the same data 

XGBoost classifier 

In [20]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Encode labels
unique_labels = sorted(set(US_Y_train) | set(US_Y_test))
le = LabelEncoder()
le.fit(unique_labels)

US_Y_train_encoded = le.transform(US_Y_train)
US_Y_test_encoded = le.transform(US_Y_test)  # optional, in case needed later

# Define base model
xgb = XGBClassifier(objective='multi:softmax', num_class=len(unique_labels), random_state=42)

# Define hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 2, 5],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1]
}

# Perform randomized search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit on training data
random_search.fit(US_X_train, US_Y_train_encoded)

# Get best model
best_model = random_search.best_estimator_

# Predict and decode labels
y_pred_encoded = best_model.predict(US_X_test)
US_y_pred = le.inverse_transform(y_pred_encoded)

# Evaluate
print("Best Hyperparameters:", random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best Hyperparameters: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.6}


In [21]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Get unique labels and encode them
unique_labels = sorted(set(N_Y_train) | set(N_Y_test))
le = LabelEncoder()
le.fit(unique_labels)

N_Y_train_encoded = le.transform(N_Y_train)
N_Y_test_encoded = le.transform(N_Y_test)

model = XGBClassifier(objective='multi:softmax', num_class=len(unique_labels), random_state=42)
model.fit(N_X_train, N_Y_train_encoded)

y_pred_encoded = model.predict(N_X_test)
N_y_pred = le.inverse_transform(y_pred_encoded)

In [22]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Get unique labels and encode them
unique_labels = sorted(set(UP_Y_train) | set(UP_Y_test))
le = LabelEncoder()
le.fit(unique_labels)

UP_Y_train_encoded = le.transform(UP_Y_train)
UP_Y_test_encoded = le.transform(UP_Y_test)

model = XGBClassifier(objective='multi:softmax', num_class=len(unique_labels), random_state=42)
model.fit(UP_X_train, UP_Y_train_encoded)

y_pred_encoded = model.predict(UP_X_test)
UP_y_pred = le.inverse_transform(y_pred_encoded)

In [23]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Get unique labels and encode them
unique_labels = sorted(set(SM_Y_train) | set(N_Y_test))
le = LabelEncoder()
le.fit(unique_labels)

SM_Y_train_encoded = le.transform(SM_Y_train)

model = XGBClassifier(objective='multi:softmax', num_class=len(unique_labels), random_state=42)
model.fit(SM_X_train, SM_Y_train_encoded)

y_pred_encoded = model.predict(N_X_test)
SM_y_pred = le.inverse_transform(y_pred_encoded)

In [24]:
print("Accuracy for Undersampled data:", accuracy_score(US_Y_test, US_y_pred))
print(classification_report(US_Y_test, US_y_pred))
print("Accuracy for normal data:", accuracy_score(N_Y_test, N_y_pred))
print(classification_report(N_Y_test, N_y_pred))
print("Accuracy for normal data:", accuracy_score(UP_Y_test, UP_y_pred))
print(classification_report(UP_Y_test, UP_y_pred))

print("Accuracy for normal data:", accuracy_score(N_Y_test, SM_y_pred))
print(classification_report(N_Y_test, SM_y_pred))

Accuracy for Undersampled data: 0.5822916666666667
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        44
           5       0.67      0.64      0.65       396
           6       0.54      0.70      0.61       382
           7       0.48      0.33      0.39       118
           8       0.00      0.00      0.00        12

    accuracy                           0.58       960
   macro avg       0.28      0.28      0.28       960
weighted avg       0.55      0.58      0.56       960

Accuracy for normal data: 0.5010416666666667
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        44
           5       0.59      0.54      0.56       396
           6       0.47      0.62      0.53       382
           7       0.36      0.27      0.31       118
           8       0.33      0.08   

c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 1️⃣ Cross-validation on TRAINING data
train_cv_scores = cross_val_score(
    best_model,
    US_X_train,
    US_Y_train_encoded,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

print("\n💡 Cross-Validation on TRAINING Set:")
print(f"Scores: {train_cv_scores}")
print(f"Mean Accuracy: {np.mean(train_cv_scores):.4f}")
print(f"Standard Deviation: {np.std(train_cv_scores):.4f}")

# 2️⃣ Evaluation on TEST data
y_test_pred_encoded = best_model.predict(US_X_test)
y_test_pred = le.inverse_transform(y_test_pred_encoded)

print("\n🧪 Evaluation on TEST Set:")
print("Accuracy:", accuracy_score(US_Y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(US_Y_test, y_test_pred))

c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



💡 Cross-Validation on TRAINING Set:
Scores: [0.609375   0.61607143 0.640625   0.55257271 0.56823266]
Mean Accuracy: 0.5974
Standard Deviation: 0.0323

🧪 Evaluation on TEST Set:
Accuracy: 0.5822916666666667

Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        44
           5       0.67      0.64      0.65       396
           6       0.54      0.70      0.61       382
           7       0.48      0.33      0.39       118
           8       0.00      0.00      0.00        12

    accuracy                           0.58       960
   macro avg       0.28      0.28      0.28       960
weighted avg       0.55      0.58      0.56       960



c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### random forest classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [27]:
le = LabelEncoder()
US_Y_train_encoded = le.fit_transform(US_Y_train)
US_Y_test_encoded = le.transform(US_Y_test)
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(US_X_train, US_Y_train_encoded)
y_pred_encoded = rf_classifier.predict(US_X_test)
US_y_pred = le.inverse_transform(y_pred_encoded)

In [28]:
N_Y_train_encoded = le.fit_transform(N_Y_train)
N_Y_test_encoded = le.transform(N_Y_test)
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(N_X_train, N_Y_train_encoded)
y_pred_encoded = rf_classifier.predict(N_X_test)
N_y_pred = le.inverse_transform(y_pred_encoded)

In [29]:
print("Accuracy for Undersampled data:", accuracy_score(US_Y_test, US_y_pred))
print(classification_report(US_Y_test, US_y_pred))
print("Accuracy for normal data:", accuracy_score(N_Y_test, N_y_pred))
print(classification_report(N_Y_test, N_y_pred))

Accuracy for Undersampled data: 0.6229166666666667
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         8
           4       0.50      0.02      0.04        44
           5       0.73      0.68      0.70       396
           6       0.56      0.75      0.64       382
           7       0.52      0.36      0.42       118
           8       1.00      0.08      0.15        12

    accuracy                           0.62       960
   macro avg       0.55      0.31      0.33       960
weighted avg       0.62      0.62      0.60       960

Accuracy for normal data: 0.5197916666666667
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        44
           5       0.64      0.53      0.58       396
           6       0.47      0.70      0.57       382
           7       0.34      0.19      0.25       118
           8       0.00      0.00   

c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo